In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [3]:
from train_sentences import TRAIN_DATA 


import random
import spacy
from spacy.training import Example


In [4]:
# Загрузка пустой модели с поддержкой русского языка
nlp = spacy.blank('ru')

# Создание нового компонента NER и добавление его в конвейер
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

# Добавление новых меток в NER
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# Начало обучения
nlp.begin_training()
for itn in range(10):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.2, losses=losses)

d:\Programming\Python\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "на предыдущую субботу зафиксированный пассажиропот..." with entities "[(3, 21, 'DATE'), (56, 64, 'STATION'), (65, 87, 'L...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Programming\Python\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "сколько людей проезжает через багратионовскую филе..." with entities "[(30, 45, 'STATION'), (46, 55, 'LINE'), (68, 75, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Programming\Python\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be 

In [5]:
def process_query(query):
    doc = nlp(query)
    result = {}
    for ent in doc.ents:
        result[ent.label_] = ent.text
    return result

In [10]:
query = 'сколько людей было на 14 мая на мичуринском проспекте большая кольцевая линии'
process_query(query)

{'DATE': '14 мая',
 'STATION': 'мичуринском проспекте',
 'LINE': 'большая кольцевая'}

# Сохранение модели в файл

In [11]:
nlp.to_disk("sentence_model")